__体素缺陷人工标注代码__

In [ ]:
import numpy as np
import scipy.io as io

data = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat')
data = data['defectdata']
data = np.array(data)
index = io.loadmat('../../Matlab_files/index.mat')
index = index['indexdata1']
index = np.array(index)
index_array = []
for i in range(len(index)):
    #提取一行进行处理
    deal_index = index[i]

    
    #增加维度，调整x、y、z列的顺序
    deal_index = deal_index[np.newaxis,:]
    deal_index = deal_index[:,[2, 0, 1]]

    #matlab中坐标转化到python中坐标
    new_x = 120 - deal_index[:,[0]]
    new_z = 120 - deal_index[:,[1]]
    new_y =  -deal_index[:,[2]]
    
    #将坐标存储到坐标列表中
    temp_array = [new_x, new_z, new_y]
    temp_array = np.squeeze(temp_array)
    index_array.append(temp_array)

index_array = np.array(index_array)
print(index_array)

for t in range(len(index_array)):
    embed_index = index_array[t]
    print(embed_index)
    x = embed_index[0]
    z = embed_index[1]
    y = embed_index[2]
    data[x][z][y] = 1

io.savemat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat',{'defectdata':data})


__数据集切割代码__

In [4]:
import numpy as np
import scipy.io as io
import os

def cutmodule(matdata, saveindex, signal):
    if signal == True:
        data = matdata['data']
        print(data.shape)
        print('****************执行数据尺寸修改****************')
        nelx, nely, nelz = 120, 120, 120
        if (nelx - data.shape[0])%2 !=0:
            data = np.pad(data, ((1, 0), (0, 0), (0 ,0)))
        if (nelz - data.shape[1])%2 !=0:
            data = np.pad(data, ((0, 0), (1, 0), (0 ,0)))
        if (nely - data.shape[2])%2 !=0:
            data = np.pad(data, ((0, 0), (0, 0), (1 ,0)))
        data = np.pad(data, (((nelx - data.shape[0])//2, (nelx - data.shape[0])//2),
                        ((nely - data.shape[1])//2, (nely - data.shape[1])//2),
                        ((nelz - data.shape[2])//2, (nelx - data.shape[2])//2)))
        io.savemat(os.path.join(saveindex,'inputdata.mat'),{'inputdata':data})
        tempdata = np.zeros((20, 20, 20))
        for k in range(0, data.shape[2], 20):               #沿y轴切割
            for j in range(0, data.shape[1], 20):           #沿z轴切割
                for i in range(0, data.shape[0], 20):       #沿x轴切割
                    global m
                    m += 1
                    tempdata = data[i:i+20, j:j+20, k:k+20]
                    # np.set_printoptions(threshold=np.inf)
                    np.save(os.path.join(saveindex,'%d.npy'%m),tempdata)
                    print('第：',m,'次')
                    # print(tempdata)
                    tempdata = np.zeros((20, 20, 20))
    else:
        data = matdata['defectdata']
        print(data.shape)
        print('****************生成标签数据集****************')
        tempdata = np.zeros((20, 20, 20))
        for k in range(0, data.shape[2], 20):               #沿y轴切割
            for j in range(0, data.shape[1], 20):           #沿z轴切割
                for i in range(0, data.shape[0], 20):       #沿x轴切割
                    global s
                    s += 1
                    tempdata = data[i:i+20, j:j+20, k:k+20]
                    # np.set_printoptions(threshold=np.inf)
                    np.save(os.path.join(saveindex,'%d.npy'%s),tempdata)
                    print('第：',s,'次')
                    # print(tempdata)
                    tempdata = np.zeros((20, 20, 20))

__非零矩阵标号检测__

In [5]:
import numpy as np
import os
import glob

def nozerosdetect(data_index):

    global m
    global s
    n = 0
    zeros_array = []
    array = np.arange((m-215), (m+1))
    
    for i in range((m-215),(m+1)):
        path = os.path.join(data_index, str(i)+'.npy')
        data = np.load(path)
        if np.all(data == 0):
            n += 1
            zeros_array.append(i)
        else:
            zeros_array.append(0)

    print('一共',n,'个全零矩阵')

    zeros_array = np.array(zeros_array)
    nozeros_array = array - zeros_array
    np.save(os.path.join(data_index, 'detectmatrix.npy'), nozeros_array)

__主函数__

In [6]:
import numpy as np
import os
import scipy.io as io
import glob

folder_address = '../Matlab_files/dataset/originalmatdata/dataset/generatedata/input/'
mat_address = '../Matlab_files/dataset/originalmatdata/matdata/input/'
#test
# folder_address = '../Matlab_files/dataset/originalmatdata/testdataset/input/'
# mat_address = '../Matlab_files/dataset/originalmatdata/matdata/input/'

matdata_index = glob.glob(os.path.join(mat_address,'*.mat')) 

m = 0
s = 0
for t in range(0, len(matdata_index)):

    print('读取第',t,'个数据')


    #输入数据集生成
    os.makedirs(os.path.join(folder_address,str(t)))
    inputpath = os.path.join(mat_address,str(t)+'.mat')      #按顺序读取输入数据
    inputmatdata = io.loadmat(inputpath)
    saveinputindex = os.path.join(folder_address,str(t))
    cutmodule(inputmatdata, saveinputindex, signal=True)
    data_index = os.path.join('../Matlab_files/dataset/originalmatdata/dataset/generatedata/input/',str(t)+'/')
    nozerosdetect(data_index)
    
    
    #标签数据集生成
    label_folder_address = folder_address.replace('input/','label/')   #文件夹生成地址
    os.makedirs(os.path.join(label_folder_address,str(t)))             #生成对应模型的存储文件夹
    labelpath = inputpath.replace('input/','label/')                   #标签数据读取路径
    savelabelindex = os.path.join(label_folder_address,str(t))
    labelmatdata = io.loadmat(labelpath)
    cutmodule(labelmatdata, savelabelindex, signal=False)
    label_data_index = data_index.replace('input/','label/')
    nozerosdetect(label_data_index)
    print('*'*30)
    


读取第 0 个数据
(20, 60, 16)
****************执行数据尺寸修改****************
第： 1 次
第： 2 次
第： 3 次
第： 4 次
第： 5 次
第： 6 次
第： 7 次
第： 8 次
第： 9 次
第： 10 次
第： 11 次
第： 12 次
第： 13 次
第： 14 次
第： 15 次
第： 16 次
第： 17 次
第： 18 次
第： 19 次
第： 20 次
第： 21 次
第： 22 次
第： 23 次
第： 24 次
第： 25 次
第： 26 次
第： 27 次
第： 28 次
第： 29 次
第： 30 次
第： 31 次
第： 32 次
第： 33 次
第： 34 次
第： 35 次
第： 36 次
第： 37 次
第： 38 次
第： 39 次
第： 40 次
第： 41 次
第： 42 次
第： 43 次
第： 44 次
第： 45 次
第： 46 次
第： 47 次
第： 48 次
第： 49 次
第： 50 次
第： 51 次
第： 52 次
第： 53 次
第： 54 次
第： 55 次
第： 56 次
第： 57 次
第： 58 次
第： 59 次
第： 60 次
第： 61 次
第： 62 次
第： 63 次
第： 64 次
第： 65 次
第： 66 次
第： 67 次
第： 68 次
第： 69 次
第： 70 次
第： 71 次
第： 72 次
第： 73 次
第： 74 次
第： 75 次
第： 76 次
第： 77 次
第： 78 次
第： 79 次
第： 80 次
第： 81 次
第： 82 次
第： 83 次
第： 84 次
第： 85 次
第： 86 次
第： 87 次
第： 88 次
第： 89 次
第： 90 次
第： 91 次
第： 92 次
第： 93 次
第： 94 次
第： 95 次
第： 96 次
第： 97 次
第： 98 次
第： 99 次
第： 100 次
第： 101 次
第： 102 次
第： 103 次
第： 104 次
第： 105 次
第： 106 次
第： 107 次
第： 108 次
第： 109 次
第： 110 次
第： 111 次
第： 112 次
第： 113 次
第： 114 次
第： 115 次
第： 116 次


In [10]:
import numpy as np
import scipy.io as io
# data = np.load('../Matlab_files/dataset/originalmatdata/dataset/formaldata/input/0/detectmatrix.npy')
# print(data)
inputdata = np.load('../Matlab_files/dataset/originalmatdata/dataset/formaldata/input/81.npy')
print(inputdata.shape)
io.savemat('./testdata.mat',{'inputdata':inputdata})
labeldata = np.load('../Matlab_files/dataset/originalmatdata/dataset/formaldata/label/81.npy')
io.savemat('./labeldata.mat',{'labeldata':labeldata})

(20, 20, 20)


In [2]:
import numpy as np
import scipy.io as io

data = io.loadmat('../Matlab_files/dataset/originalmatdata/matdata/input/0.mat')
data = data['data']
print(data.shape)

(20, 60, 16)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
data = np.load('../Matlab_files/dataset/originalmatdata/dataset/label/45/detectmatrix.npy')
print(data)
slicedata = np.load('../Matlab_files/dataset/originalmatdata/dataset/label/45/9807.npy')
np.set_printoptions(threshold=np.inf)
# print(slicedata)
print(np.all(slicedata == 0))
mycolormap = plt.get_cmap('winter')
slicedata_max = slicedata.max()
relativevalue = np.round(slicedata / slicedata_max, 1)
colorsvalues = mycolormap(relativevalue)
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(projection='3d')
ax.set_xlabel('X Axis')
ax.set_ylabel('Y Axis')
ax.set_zlabel('Z Axis')
ax.set_xlim(0, 20)
ax.set_ylim(0, 20)
ax.set_zlim(10, -10)
ax.voxels(slicedata, facecolors=colorsvalues,edgecolors = 'k', shade = False)

__Mask转One-hot__

In [ ]:
import torch
index = torch.randint(7 ,size=(1, 1, 7, 7)).long()
print(index)
x_onehot = torch.zeros(1, 7, 7, 7).long()
x_onehot.scatter_(1, index, 1).float()
print(x_onehot.numpy())

__defectdata实现Mask转One-hot__

In [ ]:
import scipy.io as io
import numpy as np
import torch
matdata = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat')
data = matdata['defectdata']
np.set_printoptions(threshold=np.inf)
# print(data)
data = data[np.newaxis,np.newaxis,:,:,:]
data = torch.from_numpy(data).long()
print(data.shape)
np.set_printoptions(threshold=np.inf)
output = torch.zeros(1, 3, 120, 120, 120).long()            #output的channel数一定要等于类别数，同时也要等于mask中的类别数，因为scatter相当于是Mask
                                                            #拆成了与类别数相对应的One-hot，One-hot每个通道对应一个类别，总之：
                                                            #One-hot channel数 = 类别数 = mask中数字类别个数
print(output.shape)
output = output.scatter(1, data, 1)
print(output)

In [ ]:
import scipy.io as io
import numpy as np

# matdata = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat')
matdata = io.loadmat('./test.mat')
data = matdata['data']
print(data.shape)
b = np.nonzero(data)
b = np.array(b)
print(b)
print(type(b))

for i in range(b.shape[1]):
    print('第',i,'个数据')
    print(data[(b[0])[i]][(b[1])[i]][(b[2])[i]])
    m = i
print(m)

In [ ]:
import torch

index = torch.randint(3, (1, 1, 5, 5)).long()
print('index:\n',index)

ouput_onehot = torch.zeros(1, 3, 5, 5).long()
ouput_onehot = ouput_onehot.scatter(1, index, 1)
print('output_onehot:\n',ouput_onehot)


In [ ]:
import scipy.io as io
import numpy as np

matdata = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat')
data = matdata['defectdata']
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        for k in range(data.shape[2]):
            if data[i][j][k] != 0:
                data[i][j][k] = 1
            
io.savemat('test.mat',{'data':data})

In [ ]:
import numpy as np
import scipy.io as io
import torch
matdata = io.loadmat('./test.mat')
data = matdata['data']
data = data[np.newaxis,np.newaxis,:,:,:]
data = torch.from_numpy(data).long()

print(data.shape)

output_onehot = torch.zeros(1, 2, 120, 120, 120)
output_onehot = output_onehot.scatter(1 ,data, 1)
print(output_onehot)

In [ ]:
import numpy as np

data_index = np.load('../../Matlab_files/dataset/originalmatdata/dataset/input/0/detectmatrix.npy')
data = np.load('../../Matlab_files/dataset/originalmatdata/dataset/input/0/82.npy')
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        for k in range(data.shape[2]):
            if data[i][j][k] != 0:
                data[i][j][k] = 1
np.set_printoptions(threshold=np.inf)
# print(data)
# print(data_index)
# print(data[2,:,:])
array = np.argmax(data, axis = 0)
print(array)
print(array.shape)

In [ ]:
import numpy as np
import scipy.io as io
data = np.load('../Matlab_files/dataset/originalmatdata/dataset/generatedata/label/0/detectmatrix.npy')
print(data)
# labeldata = np.load('../Matlab_files/dataset/originalmatdata/dataset/generatedata/label/20/4414.npy')
# inputdata = np.load('../Matlab_files/dataset/originalmatdata/dataset/generatedata/input/20/4414.npy')

# io.savemat('./labeldata.mat',{'labeldata':labeldata})
# io.savemat('./inputdata.mat',{'inputdata':inputdata})

In [11]:
import numpy as np

data = np.load('../Matlab_files/dataset/originalmatdata/dataset/formaldata/input/82.npy')
np.set_printoptions(threshold=np.inf)
# print(data)
print(data.shape)
labeldata = np.load('../Matlab_files/dataset/originalmatdata/dataset/formaldata/label/1.npy')
print('labeldata:',labeldata.shape)

(20, 20, 20)
labeldata: (20, 20, 20)


In [23]:
import numpy as np
import glob
import os 

inputdataaddres = '../Matlab_files/dataset/originalmatdata/dataset/formaldata/label/*.npy'
inputdataindex = glob.glob(inputdataaddres)
print(len(inputdataindex))
for i in range(len(inputdataindex)):
    print('第',i,'个数据')
    data = np.load(inputdataindex[i])
    print('修改前维度：',data.shape)
    data = data[np.newaxis,:,:,:]
    print('修改后维度：',data.shape)
    np.save(os.path.join('../Matlab_files/dataset/dataset（2023.05.04）/label/','%d.npy'%i),data)

9936
第 0 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 1 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 2 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 3 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 4 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 5 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 6 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 7 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 8 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 9 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 10 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 11 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 12 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 13 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 14 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 15 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 16 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 17 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 18 个数据
修改前维度： (20, 20, 20)
修改后维度： (1, 20, 20, 20)
第 19 个数据
修改前维度： (

In [24]:
import numpy as np

data = np.load('../Matlab_files/dataset/dataset（2023.05.04）/label/0.npy')
print(data.shape)

(1, 20, 20, 20)


In [1]:
import math
beta = 0.9
confusion = [5.6088358e+00, 2.6218030e-01, 1.4923610e-01, 3.3700470e-01,
       3.4648100e-02, 1.9298751e+00, 9.6757000e-03, 2.7778000e-03]
for i in confusion:
    weight = [((1.0 - beta) / (1.0 - math.pow(beta, i)))]
    print(weight)

[0.22411525748531982]
[3.67034260408721]
[6.410000780100886]
[2.8666428327532065]
[27.44322537567359]
[0.543498238277796]
[98.14339275721821]
[341.73124590610587]


In [ ]:
import torch

def cb_loss(y_pred, y_true, temp, beta, confusion):
    confusion_ = [torch.sum(temp == i) for i in range(8)]

    weight = []
    weight_dice = []
    for i, n in zip(confusion_, confusion):
        if i == 0:
            weight.append(0)
            weight_dice.append(1)
        else:
            weight_dice.append(1)
            weight.append(((1.0 - beta) / (1.0 - math.pow(beta, n))))
    weight = torch.FloatTensor(weight)
    weight_dice = torch.FloatTensor(weight_dice)
    weight = weight.to(y_pred.device)
    weight_dice = weight_dice.to(y_pred.device)

    criterion_train = torch.nn.CrossEntropyLoss(weight=weight)

    loss = criterion_train(y_pred.float(), temp.float())

    return loss

In [3]:
confusion = [1, 2, 3, 4]
confusion_ = [5, 6, 7, 8]
for i, n in zip(confusion_, confusion):
    print('i',i)
    print('n',n)
    print(list(zip(confusion_, confusion)))

i 5
n 1
[(5, 1), (6, 2), (7, 3), (8, 4)]
i 6
n 2
[(5, 1), (6, 2), (7, 3), (8, 4)]
i 7
n 3
[(5, 1), (6, 2), (7, 3), (8, 4)]
i 8
n 4
[(5, 1), (6, 2), (7, 3), (8, 4)]


In [4]:
import numpy as np
x = np.array([-2, 3, -9.4, 5, 0])
ex = np.sum(np.exp(x)/np.sum(np.exp(x))*np.array([0, 1, 2, 3, 4]))
print(ex)

2.766691018220332


In [12]:
import torch
import torch.nn.functional as F
x = torch.randn(5, 5)
print(x)
target = torch.tensor([0, 2, 3, 1, 4])
one_hot = F.one_hot(target).float()
print(one_hot)
softmax = torch.exp(x)/torch.sum(torch.exp(x),dim=1).reshape(-1, 1)
print(softmax)
logsoftmax = torch.log(softmax)
print(logsoftmax)
nllloss = -torch.sum(one_hot*logsoftmax)/target.shape[0]
print(nllloss)

tensor([[-0.0567,  0.9720,  1.1253, -0.2701, -0.0982],
        [ 0.2043, -0.0274,  0.9699, -2.4157,  0.1215],
        [-0.7145, -0.2380, -1.6991,  1.6686, -0.3015],
        [ 1.2556, -0.7402, -0.8204,  0.3770, -0.8827],
        [ 0.6922,  1.3716, -1.3396, -0.1213, -0.3449]])
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])
tensor([[0.1133, 0.3170, 0.3695, 0.0915, 0.1087],
        [0.2026, 0.1607, 0.4356, 0.0147, 0.1865],
        [0.0652, 0.1050, 0.0244, 0.7068, 0.0986],
        [0.5572, 0.0757, 0.0699, 0.2315, 0.0657],
        [0.2563, 0.5056, 0.0336, 0.1136, 0.0909]])
tensor([[-2.1776, -1.1489, -0.9956, -2.3911, -2.2192],
        [-1.5967, -1.8284, -0.8311, -4.2167, -1.6795],
        [-2.7300, -2.2536, -3.7147, -0.3470, -2.3171],
        [-0.5848, -2.5805, -2.6607, -1.4634, -2.7231],
        [-1.3614, -0.6820, -3.3932, -2.1749, -2.3985]])
tensor(1.6670)


In [ ]:
import numpy as np
import os
import glob

def nozerosdetect(data_index):

    global m
    global s
    n = 0
    zeros_array = []
    array = np.arange((m-215), (m+1))
    
    for i in range((m-215),(m+1)):
        path = os.path.join(data_index, str(i)+'.npy')
        data = np.load(path)
        if np.all(data == 0):
            n += 1
            zeros_array.append(i)
        else:
            zeros_array.append(0)

    print('一共',n,'个全零矩阵')

    zeros_array = np.array(zeros_array)
    nozeros_array = array - zeros_array
    np.save(os.path.join(data_index, 'detectmatrix.npy'), nozeros_array)